# Twitter Trust Correlation

In [0]:
import pandas as pd
import numpy as np
import pickle as pk

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [3]:
!wget http://muftayebaly.com/dropbox/tweetsCID.csv

--2020-04-15 19:01:22--  http://muftayebaly.com/dropbox/tweetsCID.csv
Resolving muftayebaly.com (muftayebaly.com)... 107.180.20.85
Connecting to muftayebaly.com (muftayebaly.com)|107.180.20.85|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 54234968 (52M) [text/csv]
Saving to: ‘tweetsCID.csv’

tweetsCID.csv       100%[===================>]  51.72M  14.3MB/s    in 4.2s    

2020-04-15 19:01:27 (12.5 MB/s) - ‘tweetsCID.csv’ saved [54234968/54234968]



In [4]:
!wget http://muftayebaly.com/dropbox/all-user-stats.csv

--2020-04-15 19:02:14--  http://muftayebaly.com/dropbox/all-user-stats.csv
Resolving muftayebaly.com (muftayebaly.com)... 107.180.20.85
Connecting to muftayebaly.com (muftayebaly.com)|107.180.20.85|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 61002615 (58M) [text/csv]
Saving to: ‘all-user-stats.csv’

all-user-stats.csv  100%[===================>]  58.18M  17.2MB/s    in 4.6s    

2020-04-15 19:02:19 (12.7 MB/s) - ‘all-user-stats.csv’ saved [61002615/61002615]



In [5]:
!wget https://raw.githubusercontent.com/harish-cu/tweet-url-relationships/master/data/raw/knight_data_articles.csv

--2020-04-15 19:02:28--  https://raw.githubusercontent.com/harish-cu/tweet-url-relationships/master/data/raw/knight_data_articles.csv
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 151.101.0.133, 151.101.64.133, 151.101.128.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|151.101.0.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 9186850 (8.8M) [text/plain]
Saving to: ‘knight_data_articles.csv’

knight_data_article 100%[===================>]   8.76M  --.-KB/s    in 0.05s   

2020-04-15 19:02:29 (164 MB/s) - ‘knight_data_articles.csv’ saved [9186850/9186850]



In [6]:
!wget https://raw.githubusercontent.com/harish-cu/tweet-url-relationships/master/data/raw/knight_data_urls.csv

--2020-04-15 19:02:34--  https://raw.githubusercontent.com/harish-cu/tweet-url-relationships/master/data/raw/knight_data_urls.csv
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 151.101.0.133, 151.101.64.133, 151.101.128.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|151.101.0.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 9199526 (8.8M) [text/plain]
Saving to: ‘knight_data_urls.csv’

knight_data_urls.cs 100%[===================>]   8.77M  --.-KB/s    in 0.1s    

2020-04-15 19:02:36 (59.4 MB/s) - ‘knight_data_urls.csv’ saved [9199526/9199526]



In [7]:
!wget https://raw.githubusercontent.com/harish-cu/tweet-url-relationships/master/data/raw/knight_data_ratings.csv

--2020-04-15 19:02:40--  https://raw.githubusercontent.com/harish-cu/tweet-url-relationships/master/data/raw/knight_data_ratings.csv
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 151.101.0.133, 151.101.64.133, 151.101.128.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|151.101.0.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 2344804 (2.2M) [text/plain]
Saving to: ‘knight_data_ratings.csv’

knight_data_ratings 100%[===================>]   2.24M  --.-KB/s    in 0.07s   

2020-04-15 19:02:40 (30.3 MB/s) - ‘knight_data_ratings.csv’ saved [2344804/2344804]



In [8]:
!wget https://raw.githubusercontent.com/harish-cu/tweet-url-relationships/master/data/processed/article_topics_6.csv

--2020-04-15 19:02:43--  https://raw.githubusercontent.com/harish-cu/tweet-url-relationships/master/data/processed/article_topics_6.csv
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 151.101.0.133, 151.101.64.133, 151.101.128.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|151.101.0.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 6035965 (5.8M) [text/plain]
Saving to: ‘article_topics_6.csv’

article_topics_6.cs 100%[===================>]   5.76M  --.-KB/s    in 0.1s    

2020-04-15 19:02:44 (46.4 MB/s) - ‘article_topics_6.csv’ saved [6035965/6035965]



In [9]:
raw_topics = pd.read_csv("article_topics_6.csv")
topics = raw_topics.set_index(['content_id'])
topics.head()

,Unnamed: 0,content_body_clean,content_title_clean,topic,topic_id1,topic_p1,topic_id2,topic_p2,topic_dist
content_id,,,,,,,,,
60,0,republican congress make break moment everyone...,tax reform failure act could mean minority sta...,Policy,0,0.548654,4.0,0.398622,"[(0, 0.54865676), (1, 0.035524607), (2, 0.0146..."
61,1,constitution authorize congress tax american p...,tax reform america need tax code good country ...,Policy,0,0.832200,3.0,0.098822,"[(0, 0.83220094), (3, 0.098824225), (4, 0.0664..."
66,2,longmothballed unfinished casinohotel la vega ...,longmothballed la vega strip casinohotel sell,Economy,2,0.677059,3.0,0.147280,"[(1, 0.09982404), (2, 0.6770535), (3, 0.147273..."
80,3,tax foreign earnings already accumulate overse...,trump tax plan may free corporate dollar,Policy,0,0.994644,NaN,NaN,"[(0, 0.9946441)]"
82,4,hurricane harvey already take enormous human t...,hurricane harvey humanitarian disaster also se...,Public Health,6,0.759787,2.0,0.237297,"[(2, 0.23729898), (6, 0.7597849)]"


In [0]:
raw_data = pd.read_csv("knight_data_articles.csv")
#raw_data.head()

In [23]:
articles = raw_data.set_index(['content_id'])
articles

,content_body_clean,content_title_clean,label,economy,topic2,science,mention_Trump,mention_Clinton,mention_Democrat,mention_Republican,mention_GOP,mention_Administration,mention_political,mention_voters,mention_campaign,mention_climate,mention_abortion,mention_immigration,mention_trade,mention_immigrants,mention_border,mention_terrorist,mention_attack,mention_NRA,mention_sanctuary,mention_socialist,mention_fascist,mention_communist,mention_nationalist,mention_2A,content_source_desc
content_id,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
60,Republicans in Congress are at a make or break...,Tax reform now -- failure to act could mean mi...,9,1,0,0,1,0,0,1,1,0,1,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,Fox News
61,The Constitution authorizes Congress to tax Am...,Tax reform -- America needs a tax code that's ...,9,1,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,Fox News
66,"A long-mothballed, unfinished casino-hotel on ...",Long-mothballed Las Vegas Strip casino-hotel s...,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,Fox News
80,If taxing foreign earnings that have already a...,"Trump Tax Plan May Free Up Corporate Dollars, ...",3,1,0,0,1,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,The New York Times
82,Hurricane Harvey has already taken an enormous...,Hurricane Harvey is a humanitarian disaster. I...,5,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,Vox
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3380,"In this Oct. 17, 2017 file photo, Venezuela's...",Investors leave Venezuela meeting with no clea...,2,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,Associated Press News
3381,A man watches stock prices boards at a brokera...,Global stocks turn lower after recent record h...,6,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,Associated Press News
3382,The draining of a massive aquifer that underl...,Rural areas at risk as water levels drop in ma...,5,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,Associated Press News


In [0]:
raw_urls = pd.read_csv("knight_data_urls.csv")
#raw_urls.head()

In [13]:
raw_tweets = pd.read_csv("tweetsCID.csv")
raw_tweets

,tweet_id,tweet_text,user_id,tweet_time,no_favs,no_retweets,urls,content_id
0,851485433003421701,#Trump’s border wall would be an ecological an...,613521891,2017-04-10T17:22:01.000Z,0,0,https://www.vox.com/energy-and-environment/201...,2736
1,851514117374398464,#Trump’s border wall would be an ecological an...,222481411,2017-04-10T19:16:00.000Z,0,0,https://www.vox.com/energy-and-environment/201...,2736
2,855009400670617600,The research and experts are clear: Tougher bo...,2347049341,2017-04-20T10:45:01.000Z,0,0,https://www.vox.com/policy-and-politics/2017/4...,2644
3,855009683542728704,Hmmmmmm https://t.co/y3fNx7KuJT,243580387,2017-04-20T10:46:08.000Z,0,0,https://www.vox.com/policy-and-politics/2017/4...,2644
4,855011873183342592,(psst! It was never about drugs. It's always b...,17889654,2017-04-20T10:54:50.000Z,0,0,https://www.vox.com/policy-and-politics/2017/4...,2644
...,...,...,...,...,...,...,...,...
201868,930566176907853824,Big increase in investment hastening arrival o...,454368263,2017-11-14T22:40:40.000Z,0,0,https://www.nytimes.com/2017/11/13/business/se...,3351
201869,930571261775876097,As readers of https://t.co/ajmt36bUQU already ...,61681958,2017-11-14T23:00:52.000Z,0,0,https://tytnetwork.com/Investigates/; https://...,3351
201870,930573435343908864,We’ve been reporting on what FedEx and UPS are...,905632398511046656,2017-11-14T23:09:30.000Z,0,0,https://www.nytimes.com/2017/11/13/business/se...,3351
201871,930582220800233473,Self-Driving Trucks May Be Closer Than They Ap...,831215040,2017-11-14T23:44:25.000Z,0,0,https://www.nytimes.com/2017/11/13/business/se...,3351


In [14]:
raw_tweets.describe()

,tweet_id,user_id,no_favs,no_retweets
count,2.018730e+05,2.018730e+05,201873.0,201873.0
mean,9.207283e+17,1.332248e+17,0.0,0.0
std,7.496690e+15,3.013590e+17,0.0,0.0
min,8.514854e+17,5.700000e+01,0.0,0.0
25%,9.174132e+17,5.645870e+07,0.0,0.0
50%,9.218926e+17,4.205027e+08,0.0,0.0
75%,9.260537e+17,2.853383e+09,0.0,0.0
max,9.305854e+17,9.301665e+17,0.0,0.0


In [15]:
raw_ratings = pd.read_csv("knight_data_ratings.csv")
raw_ratings.head()

,external_user_id,content_id,rating_scale_response,sampdate,blind
0,047641_S02_01,60,4.0,20170901,0
1,4205985366_78419_01,60,5.0,20170901,1
2,4205985366_78419_01,61,4.5,20170901,1
3,4205985366_78419_01,66,3.5,20170901,1
4,4205985366_78419_01,80,4.0,20170901,1


In [16]:
raw_users = pd.read_csv("all-user-stats.csv")
raw_users['content_id']=raw_tweets['content_id']
raw_users

,tweet_id,tweet_text,user_id,tweet_time,no_favs,no_retweets,user_friends,user_followers,user_statuses,user_listed,urls,content_id
0,851485433003421701,#Trump’s border wall would be an ecological an...,613521891,2017-04-10T17:22:01.000Z,0,0,1151,1514,2649,65,https://www.vox.com/energy-and-environment/201...,2736
1,851514117374398464,#Trump’s border wall would be an ecological an...,222481411,2017-04-10T19:16:00.000Z,0,0,3252,22924,13692,698,https://www.vox.com/energy-and-environment/201...,2736
2,855009400670617600,The research and experts are clear: Tougher bo...,2347049341,2017-04-20T10:45:01.000Z,0,0,212,570484,57251,12700,https://www.vox.com/policy-and-politics/2017/4...,2644
3,855009683542728704,Hmmmmmm https://t.co/y3fNx7KuJT,243580387,2017-04-20T10:46:08.000Z,0,0,300,6276,118715,257,https://www.vox.com/policy-and-politics/2017/4...,2644
4,855011873183342592,(psst! It was never about drugs. It's always b...,17889654,2017-04-20T10:54:50.000Z,0,0,457,1643,16647,80,https://www.vox.com/policy-and-politics/2017/4...,2644
...,...,...,...,...,...,...,...,...,...,...,...,...
201868,930566176907853824,Big increase in investment hastening arrival o...,454368263,2017-11-14T22:40:40.000Z,0,0,141,340,211,5,https://www.nytimes.com/2017/11/13/business/se...,3351
201869,930571261775876097,As readers of https://t.co/ajmt36bUQU already ...,61681958,2017-11-14T23:00:52.000Z,0,0,3190,3531,17059,121,https://tytnetwork.com/Investigates/; https://...,3351
201870,930573435343908864,We’ve been reporting on what FedEx and UPS are...,905632398511046656,2017-11-14T23:09:30.000Z,0,0,128,1555,115,19,https://www.nytimes.com/2017/11/13/business/se...,3351
201871,930582220800233473,Self-Driving Trucks May Be Closer Than They Ap...,831215040,2017-11-14T23:44:25.000Z,0,0,482,3903,9689,26,https://www.nytimes.com/2017/11/13/business/se...,3351


In [0]:
users = raw_users.set_index(['content_id'])

In [0]:
import os
import shutil

## Correlation of \# of Tweets and Trust

In [19]:
blind_ratings = raw_ratings[raw_ratings['blind']==1].groupby(['content_id']).mean()
blind_ratings['content_id1'] = blind_ratings.index
blind_ratings

,rating_scale_response,sampdate,blind,content_id1
content_id,,,,
60,5.000,20170901,1,60
61,4.500,20170901,1,61
66,3.500,20170901,1,66
80,3.750,20170901,1,80
82,3.500,20170901,1,82
...,...,...,...,...
3375,2.500,20170901,1,3375
3378,4.625,20170901,1,3378
3380,3.500,20170901,1,3380


In [20]:
num_tweets = raw_tweets.groupby(['content_id']).count()
num_tweets['content_id1'] = num_tweets.index
num_tweets

,tweet_id,tweet_text,user_id,tweet_time,no_favs,no_retweets,urls,content_id1
content_id,,,,,,,,
101,33,33,33,33,33,33,33,101
1019,14,14,14,14,14,14,14,1019
102,20,20,20,20,20,20,20,102
103,18,18,18,18,18,18,18,103
1060,2,2,2,2,2,2,2,1060
...,...,...,...,...,...,...,...,...
984,14,14,14,14,14,14,14,984
985,273,273,273,273,273,273,273,985
990,8,8,8,8,8,8,8,990


In [21]:
avg_users = users.groupby(['content_id']).mean()
avg_users

,tweet_id,user_id,no_favs,no_retweets,user_friends,user_followers,user_statuses,user_listed
content_id,,,,,,,,
101,9.022349e+17,5.155158e+16,0.0,0.0,7715.060606,18253.696970,117437.090909,554.000000
1019,9.051735e+17,2.458286e+17,0.0,0.0,9989.714286,36151.071429,111649.142857,886.285714
102,9.009004e+17,2.087483e+17,0.0,0.0,12668.350000,29445.750000,156883.850000,736.600000
103,8.987590e+17,1.873380e+17,0.0,0.0,7756.777778,42129.055556,98562.166667,1207.111111
1060,9.096514e+17,4.819011e+09,0.0,0.0,33.000000,203.500000,6953.000000,25.000000
...,...,...,...,...,...,...,...,...
984,9.091715e+17,1.619142e+17,0.0,0.0,4187.285714,19435.500000,104234.500000,270.071429
985,9.092689e+17,9.176621e+16,0.0,0.0,1828.959707,158635.344322,36624.619048,1064.611722
990,8.700455e+17,1.022339e+17,0.0,0.0,507.875000,76802.750000,20290.375000,1810.750000


In [26]:
corr_df = pd.DataFrame(columns=['content_id','article_body','article_title','num_tweets','trust_mean','source','topic','avg_friends','avg_followers','avg_statuses'])

for index, row in num_tweets.iterrows():
  try:
    corr_df = corr_df.append({'content_id':index, 'article_body':articles['content_body_clean'][int(index)], 'article_title':articles['content_title_clean'][int(index)], 
                              'num_tweets':row['tweet_id'], 'trust_mean':blind_ratings['rating_scale_response'][int(index)], 
                              'source':articles['content_source_desc'][int(index)], 'topic':topics['topic'][int(index)], 
                              'avg_friends':avg_users['user_friends'][index], 'avg_followers':avg_users['user_followers'][index], 
                              'avg_statuses':avg_users['user_statuses'][index]}, ignore_index=True)
  except Exception:
    pass

corr_df

,content_id,article_body,article_title,num_tweets,trust_mean,source,topic,avg_friends,avg_followers,avg_statuses
0,101,KELLY EVANS (CO-HOST): I just want to be clear...,CNBC lets Bush's disgraced FEMA director explo...,33,4.000000,Media Matters,Public Health,7715.060606,18253.696970,117437.090909
1,1019,Fox News used an unqualified crank to push dis...,"In honor of Labor Day, Fox & Friends let an ob...",14,1.000000,Media Matters,Economy,9989.714286,36151.071429,111649.142857
2,102,"Fox cut away from report on ""life-threatening""...","Fox cut away from report on ""life-threatening""...",20,0.500000,Media Matters,Public Health,12668.350000,29445.750000,156883.850000
3,103,Sarah Wasko / Media Matters Conservative ...,Business lobbying coalition recruits right-win...,18,0.500000,Media Matters,Politics,7756.777778,42129.055556,98562.166667
4,1060,A house rests on the beach after collapsing of...,Irma’s damage a reminder of Florida economy’s ...,2,4.000000,Associated Press News,Public Health,33.000000,203.500000,6953.000000
...,...,...,...,...,...,...,...,...,...,...
1250,969,STEVE DOOCY (CO-HOST): It was a political expl...,"Fox & Friends lauds Trump's debt limit, spendi...",13,2.000000,Media Matters,Politics,2715.692308,26620.538462,48683.153846
1251,984,Time is running out for Republicans if they wa...,Newt Gingrich: Tax cuts for small business wou...,14,4.166667,Fox News,Policy,4187.285714,19435.500000,104234.500000
1252,985,"Starting with 1957, the team looked at actual ...",Bump in U.S. Incomes Doesn’t Erase 50 Years of...,273,2.750000,The New York Times,Public Health,1828.959707,158635.344322,36624.619048
1253,990,The Associated Press reported today that Donal...,Critics say Trump’s cellphone diplomacy is dan...,8,2.800000,Vox,Technology,507.875000,76802.750000,20290.375000


In [0]:
from scipy.stats.stats import pearsonr

In [0]:
corr_df1 = corr_df.astype({'num_tweets': 'int32'})

In [29]:
pearsonr(corr_df1['num_tweets'],corr_df1['trust_mean'])[0]

0.050122302409525775

In [30]:
np.corrcoef(corr_df1['num_tweets'].values.tolist(),corr_df1['trust_mean'].values.tolist())[0, 1]

0.05012230240952584

In [31]:
corr_df1

,content_id,article_body,article_title,num_tweets,trust_mean,source,topic,avg_friends,avg_followers,avg_statuses
0,101,KELLY EVANS (CO-HOST): I just want to be clear...,CNBC lets Bush's disgraced FEMA director explo...,33,4.000000,Media Matters,Public Health,7715.060606,18253.696970,117437.090909
1,1019,Fox News used an unqualified crank to push dis...,"In honor of Labor Day, Fox & Friends let an ob...",14,1.000000,Media Matters,Economy,9989.714286,36151.071429,111649.142857
2,102,"Fox cut away from report on ""life-threatening""...","Fox cut away from report on ""life-threatening""...",20,0.500000,Media Matters,Public Health,12668.350000,29445.750000,156883.850000
3,103,Sarah Wasko / Media Matters Conservative ...,Business lobbying coalition recruits right-win...,18,0.500000,Media Matters,Politics,7756.777778,42129.055556,98562.166667
4,1060,A house rests on the beach after collapsing of...,Irma’s damage a reminder of Florida economy’s ...,2,4.000000,Associated Press News,Public Health,33.000000,203.500000,6953.000000
...,...,...,...,...,...,...,...,...,...,...
1250,969,STEVE DOOCY (CO-HOST): It was a political expl...,"Fox & Friends lauds Trump's debt limit, spendi...",13,2.000000,Media Matters,Politics,2715.692308,26620.538462,48683.153846
1251,984,Time is running out for Republicans if they wa...,Newt Gingrich: Tax cuts for small business wou...,14,4.166667,Fox News,Policy,4187.285714,19435.500000,104234.500000
1252,985,"Starting with 1957, the team looked at actual ...",Bump in U.S. Incomes Doesn’t Erase 50 Years of...,273,2.750000,The New York Times,Public Health,1828.959707,158635.344322,36624.619048
1253,990,The Associated Press reported today that Donal...,Critics say Trump’s cellphone diplomacy is dan...,8,2.800000,Vox,Technology,507.875000,76802.750000,20290.375000


In [32]:
corr_df1.groupby(['source']).mean()

,num_tweets,trust_mean,avg_friends,avg_followers,avg_statuses
source,,,,,
100 Percent Fed Up,38.588235,2.222855,5078.596684,6941.932481,59601.539553
Associated Press News,4.295238,3.559105,2233.349159,48811.068800,36780.374397
Breitbart,130.113402,2.510586,4992.024438,10873.734905,74900.139012
Fox News,99.490196,3.264439,3049.896065,117718.596450,58662.712239
Media Matters,24.098039,2.572640,8431.892531,29686.102788,98459.964708
The New York Times,294.234177,3.555893,2613.652541,220921.438761,52926.282140
Vox,102.771552,3.373520,7254.542993,36208.125197,63829.565299


In [33]:
corr_df1.groupby(['topic']).mean()

,num_tweets,trust_mean,avg_friends,avg_followers,avg_statuses
topic,,,,,
Economy,87.508197,3.173621,2870.975037,161848.038625,65390.420461
Policy,183.082902,3.177887,4757.428486,88643.809879,60211.261361
Politics,123.334184,2.992726,4593.614341,100249.320937,63862.486464
Public Health,149.663830,3.543269,3773.961419,103533.417088,50560.743269
Scandal,166.871698,2.980684,4254.489063,85677.062380,62254.816997
Technology,42.541284,3.255277,4856.503906,40118.742417,63876.628291


In [34]:
print('100 Percent Fed Up:', pearsonr(corr_df[corr_df['source']=='100 Percent Fed Up'].num_tweets,
                                     corr_df[corr_df['source']=='100 Percent Fed Up'].trust_mean
                                     )[0])
print('Associated Press News:', pearsonr(corr_df[corr_df['source']=='Associated Press News'].num_tweets,
                                     corr_df[corr_df['source']=='Associated Press News'].trust_mean
                                     )[0])
print('Breitbart:', pearsonr(corr_df[corr_df['source']=='Breitbart'].num_tweets,
                                     corr_df[corr_df['source']=='Breitbart'].trust_mean
                                     )[0])
print('Fox News:', pearsonr(corr_df[corr_df['source']=='Fox News'].num_tweets,
                                     corr_df[corr_df['source']=='Fox News'].trust_mean
                                     )[0])
print('Media Matters:', pearsonr(corr_df[corr_df['source']=='Media Matters'].num_tweets,
                                     corr_df[corr_df['source']=='Media Matters'].trust_mean
                                     )[0])
print('The New York Times:', pearsonr(corr_df[corr_df['source']=='The New York Times'].num_tweets,
                                     corr_df[corr_df['source']=='The New York Times'].trust_mean
                                     )[0])
print('Vox:', pearsonr(corr_df[corr_df['source']=='Vox'].num_tweets,
                                     corr_df[corr_df['source']=='Vox'].trust_mean
                                     )[0])

100 Percent Fed Up: -0.029205437098217952
Associated Press News: -0.13419177533207938
Breitbart: -0.052556843882955256
Fox News: -0.27429417636187603
Media Matters: -0.05603416234001038
The New York Times: 0.08016191474413324
Vox: 0.03539024107968908


# Correlation of User Info and article trust![alt text](https://)

In [35]:
print("User Friends & Trust",pearsonr(corr_df1['avg_friends'],corr_df1['trust_mean'])[0])
print("User Followers & Trust",pearsonr(corr_df1['avg_followers'],corr_df1['trust_mean'])[0])

User Friends & Trust -0.10160187070124307
User Followers & Trust 0.1506684903193611


# Correlation Matrix

In [0]:
corrdf = pd.get_dummies(corr_df1, columns=['source','topic'])

In [37]:
corrdf.shape

(1255, 21)

In [38]:
corrdf.corr()

,num_tweets,trust_mean,avg_friends,avg_followers,avg_statuses,source_100 Percent Fed Up,source_Associated Press News,source_Breitbart,source_Fox News,source_Media Matters,source_The New York Times,source_Vox,topic_Economy,topic_Policy,topic_Politics,topic_Public Health,topic_Scandal,topic_Technology
num_tweets,1.000000,0.050122,-0.115024,-0.002388,-0.157941,-0.095083,-0.129956,-0.010702,-0.062423,-0.075395,0.291990,-0.053837,-0.036659,0.062027,-0.031577,0.018197,0.048276,-0.094666
trust_mean,0.050122,1.000000,-0.101602,0.150668,-0.085013,-0.289807,0.128389,-0.287808,0.058746,-0.125152,0.244536,0.109797,0.004799,0.010956,-0.113356,0.195987,-0.093542,0.032919
avg_friends,-0.115024,-0.101602,1.000000,-0.063136,0.489171,0.043952,-0.125702,0.055853,-0.128368,0.167165,-0.197613,0.275697,-0.065471,0.035867,0.034827,-0.053152,-0.008030,0.032002
avg_followers,-0.002388,0.150668,-0.063136,1.000000,0.007299,-0.133897,-0.070438,-0.183782,0.062687,-0.068384,0.382404,-0.142127,0.079772,-0.011339,0.022626,0.024286,-0.021719,-0.085790
avg_statuses,-0.157941,-0.085013,0.489171,0.007299,1.000000,-0.006198,-0.158398,0.135383,-0.020979,0.172115,-0.097502,0.034488,0.024151,-0.003150,0.049299,-0.105719,0.019498,0.022655
source_100 Percent Fed Up,-0.095083,-0.289807,0.043952,-0.133897,-0.006198,1.000000,-0.089873,-0.127183,-0.150195,-0.061215,-0.172543,-0.141642,-0.067228,-0.110628,0.265141,-0.135289,0.024736,-0.091729
source_Associated Press News,-0.129956,0.128389,-0.125702,-0.070438,-0.158398,-0.089873,1.000000,-0.129208,-0.152586,-0.062190,-0.175290,-0.143897,-0.028151,-0.025108,0.001262,0.105773,-0.050566,-0.021659
source_Breitbart,-0.010702,-0.287808,0.055853,-0.183782,0.135383,-0.127183,-0.129208,1.000000,-0.215930,-0.088007,-0.248058,-0.203634,-0.065901,-0.090638,0.025701,-0.160047,0.210818,0.040312
source_Fox News,-0.062423,0.058746,-0.128368,0.062687,-0.020979,-0.150195,-0.152586,-0.215930,1.000000,-0.103930,-0.292941,-0.240479,0.023994,0.015287,-0.058323,-0.110403,0.058980,0.125536
source_Media Matters,-0.075395,-0.125152,0.167165,-0.068384,0.172115,-0.061215,-0.062190,-0.088007,-0.103930,1.000000,-0.119394,-0.098012,0.009779,0.180744,-0.016805,-0.057410,-0.047155,-0.063474


# Modeling num of tweet prediction

In [39]:
!pip install tensorflow==2.1.0

     |████████████████████████████████| 421.8MB 24kB/s 
     |████████████████████████████████| 450kB 51.8MB/s 
     |████████████████████████████████| 3.9MB 61.3MB/s 
  Created wheel for gast: filename=gast-0.2.2-cp36-none-any.whl size=7540 sha256=d5ba1610244c855973302a3b4bc86cb709ec5fc1f6e05617012f8a5a6a906ddf
  Stored in directory: /root/.cache/pip/wheels/5c/2e/7e/a1d4d4fcebe6c381f378ce7743a3ced3699feb89bcfbdadadd
Successfully built gast
  Found existing installation: gast 0.3.3
    Uninstalling gast-0.3.3:
      Successfully uninstalled gast-0.3.3
  Found existing installation: tensorflow-estimator 2.2.0rc0
    Uninstalling tensorflow-estimator-2.2.0rc0:
      Successfully uninstalled tensorflow-estimator-2.2.0rc0
  Found existing installation: tensorboard 2.2.0
    Uninstalling tensorboard-2.2.0:
      Successfully uninstalled tensorboard-2.2.0
  Found existing installation: tensorflow 2.2.0rc2
    Uninstalling tensorflow-2.2.0rc2:
      Successfully uninstalled tensorflow-2.2.0rc

In [2]:
import tensorflow as tf
import pandas as pd
import tensorflow_hub as hub
import os
import re
import numpy as np
import pickle as pk
import string
import nltk
from nltk.corpus import stopwords
from nltk.corpus import wordnet
from nltk import word_tokenize, pos_tag
from nltk.stem import WordNetLemmatizer
from tqdm import tqdm_notebook
from collections import defaultdict
from tensorflow.keras import backend as K
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt
from scipy import stats
from sklearn.linear_model import LinearRegression, LogisticRegression, RidgeCV, LassoCV, Ridge, Lasso, SGDRegressor
from sklearn.svm import LinearSVR
from sklearn.pipeline import Pipeline, make_pipeline
from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer, TfidfVectorizer
from sklearn.model_selection import cross_val_score, GridSearchCV
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
from sklearn.experimental import enable_hist_gradient_boosting
from sklearn.ensemble import HistGradientBoostingRegressor, GradientBoostingRegressor
from sklearn.preprocessing import OneHotEncoder
import matplotlib.pyplot as plt
import seaborn as sns

/usr/local/lib/python3.6/dist-packages/statsmodels/tools/_testing.py:19: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  import pandas.util.testing as tm


In [41]:
nltk.download('wordnet')
nltk.download('punkt')
nltk.download('averaged_perceptron_tagger')
nltk.download('stopwords')

[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Unzipping corpora/wordnet.zip.
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Unzipping taggers/averaged_perceptron_tagger.zip.
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

In [0]:
def clean_data(text):
    #remove punctuation, digits, extra stuff. make lowercase
    text = text.lower()
    text = re.sub('\[.*?\]', '', text)
    text = re.sub('[%s]' % re.escape(string.punctuation), '', text)
    text = re.sub('\w*\d\w*', '', text)
    text = re.sub('[‘’“”…]', '', text)
    #lemma it - include POS tag in order to lemma it better
    tag_map = defaultdict(lambda : wordnet.NOUN)
    tag_map['J'] = wordnet.ADJ
    tag_map['V'] = wordnet.VERB
    tag_map['R'] = wordnet.ADV
    lemmatizer = WordNetLemmatizer()
    textTokens = word_tokenize(text)
    #remove stopwords
    word_tokens_nostop = [w for w in textTokens if not w in stopwords.words('english')] 
    #now lemma
    text = [lemmatizer.lemmatize(tok, tag_map[tag[0]]) for tok, tag in pos_tag(word_tokens_nostop)]
    return " ".join(text)

In [48]:
corrdf.head()

,content_id,article_body,article_title,num_tweets,trust_mean,avg_friends,avg_followers,avg_statuses,source_100 Percent Fed Up,source_Associated Press News,source_Breitbart,source_Fox News,source_Media Matters,source_The New York Times,source_Vox,topic_Economy,topic_Policy,topic_Politics,topic_Public Health,topic_Scandal,topic_Technology
0,101,KELLY EVANS (CO-HOST): I just want to be clear...,CNBC lets Bush's disgraced FEMA director explo...,33,4.0,7715.060606,18253.696970,117437.090909,0,0,0,0,1,0,0,0,0,0,1,0,0
1,1019,Fox News used an unqualified crank to push dis...,"In honor of Labor Day, Fox & Friends let an ob...",14,1.0,9989.714286,36151.071429,111649.142857,0,0,0,0,1,0,0,1,0,0,0,0,0
2,102,"Fox cut away from report on ""life-threatening""...","Fox cut away from report on ""life-threatening""...",20,0.5,12668.350000,29445.750000,156883.850000,0,0,0,0,1,0,0,0,0,0,1,0,0
3,103,Sarah Wasko / Media Matters Conservative ...,Business lobbying coalition recruits right-win...,18,0.5,7756.777778,42129.055556,98562.166667,0,0,0,0,1,0,0,0,0,1,0,0,0
4,1060,A house rests on the beach after collapsing of...,Irma’s damage a reminder of Florida economy’s ...,2,4.0,33.000000,203.500000,6953.000000,0,1,0,0,0,0,0,0,0,0,1,0,0


In [0]:
clean_body = corrdf["article_body"].apply(clean_data)
y = corrdf['num_tweets']
sideinfo = corrdf.drop(columns=['content_id','article_body','article_title','num_tweets','avg_friends','avg_followers','avg_statuses'])

In [0]:
X_train, X_test, y_train, y_test, side_train, side_test = train_test_split(clean_body, y, sideinfo, test_size=0.2, shuffle=True, random_state=3)

In [5]:
tfidf_vec = TfidfVectorizer(ngram_range=(1,3), min_df=10)

X_train_vec = tfidf_vec.fit_transform(X_train)
X_test_vec = tfidf_vec.transform(X_test)

NameError: ignored

In [72]:
X_train_vec.toarray()

array([[0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.]])

In [0]:
X_train_full = np.hstack((X_train_vec.toarray(), side_train.to_numpy()))
X_test_full = np.hstack([X_test_vec, side_test])